In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [129]:
#reading csvs from raw data sets
pop2020_df = pd.read_csv("pop2020.csv")
pop2019_df=pd.read_csv("pop2019.csv", encoding="ISO-8859-1")
inventory_df = pd.read_csv("Inventory.csv")
rental_df = pd.read_csv("Rental_price.csv")

In [130]:
#cleaning popuation data sets (finding common cities from population data sets)
#selecting wanted data and renaming the columns from 2020 population data

pop2020_df = pop2020_df.iloc[:, 1:4]
pop2020_df=pop2020_df.rename(columns={"Name" : "City",
                                      "2020 Pop":"2020 Population"})
pop2020_df = pop2020_df.replace({"New York City": "New York"})

#selecting wanted data and renaming the columns from 2019 population data
columns = 'NAME', 'STNAME', 'POPESTIMATE2018', 'POPESTIMATE2019'
pop2019_df=pop2019_df.loc[:, columns]
pop2019_df=pop2019_df.rename(columns={"NAME" : "City",
                                      "STNAME" : "State",
                                      "POPESTIMATE2018": "2018 Population",
                                      "POPESTIMATE2019" : "2019 Population"})

#merging population data sets

pop2020_df['City'] = pop2020_df['City'] + ' city'
population_df = pd.merge(pop2019_df, pop2020_df, on=["City", "State"], how='inner')
duplicated = population_df.loc[population_df.duplicated(subset=["City", "State"]), "City"]
pop_df = population_df.drop(duplicated.index)
pop_df.reset_index(drop=True, inplace=True)
pop_df["City"] = pop_df["City"].str.replace("city", "")

#abbrev. state names
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

pop_df['State'] = pop_df['State'].map(us_state_abbrev).fillna(pop_df['State'])

#exporting data as csv file 
pop_df.to_csv("cleaned_population.csv")
pop_df

,City,State,2018 Population,2019 Population,2020 Population
0,Birmingham,AL,210487,209403,"208,940"
1,Huntsville,AL,198125,200574,"202,910"
2,Mobile,AL,189809,188720,"186,804"
3,Montgomery,AL,198899,198525,"197,282"
4,Chandler,AZ,257186,261165,"268,675"
...,...,...,...,...,...
177,Spokane,WA,219332,222081,"223,266"
178,Tacoma,WA,216111,217827,"222,603"
179,Vancouver,WA,182887,184463,"186,516"
180,Madison,WI,257854,259680,"264,030"


In [131]:
#renaming column name and selecting wanted data from inventory.csv
inventory_df = inventory_df.rename(columns={"RegionName":"City"})
inventory_df.drop(inventory_df.iloc[:, 5:124], inplace = True, axis = 1) 
inventory_df.drop(inventory_df.iloc[:, 0:2], inplace = True, axis = 1) 
del inventory_df["RegionType"]

#renaming columns and data
inventory_df['City'] = inventory_df['City'].str.split(',').str[0]
inventory_df = inventory_df.rename(columns={'StateName':'State'})
inventory_df = inventory_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

#mering with common city list 
inventory_df = pd.merge(pop_df["City"], inventory_df, on="City", how="outer")
new_inventory_df=inventory_df.dropna(how="any")
new_inventory_df.reset_index(drop=True, inplace=True)
new_inventory_df.to_csv("cleaned_inventory.csv")
new_inventory_df

,City,State,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,...,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31
0,New York,NY,400000.0,399840.0,395000.0,400000.0,414782.0,425000.0,430000.0,430000.0,...,419986.0,420000.0,423000.0,423750.0,420000.0,435000.0,434950.0,445000.0,448250.0,455000.0
1,Los Angeles,CA,599000.0,625000.0,630000.0,635000.0,650000.0,652000.0,645000.0,650000.0,...,650000.0,655000.0,655000.0,658000.0,679000.0,665000.0,650000.0,681500.0,710000.0,725000.0
2,Chicago,IL,215000.0,217500.0,230000.0,239000.0,240000.0,249900.0,243000.0,233000.0,...,231250.0,230000.0,223000.0,225000.0,248000.0,260000.0,251000.0,254000.0,262000.0,270000.0
3,Dallas,TX,246000.0,255000.0,265000.0,265000.0,275000.0,280000.0,274000.0,267000.0,...,270000.0,275000.0,265000.0,273900.0,280000.0,285000.0,280000.0,290000.0,300000.0,300000.0
4,Philadelphia,PA,220000.0,220000.0,230000.0,235000.0,242000.0,250000.0,252000.0,255000.0,...,250000.0,250000.0,239550.0,240000.0,250000.0,270000.0,261250.0,269000.0,278000.0,279900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Faribault,MN,218900.0,218500.0,194900.0,220000.0,202750.0,221000.0,222318.0,236306.0,...,237500.0,240000.0,225450.0,226200.0,255500.0,253750.0,265000.0,267983.0,280000.0,264000.0
116,Albemarle,NC,128000.0,135000.0,143000.0,155000.0,143250.0,142750.0,139000.0,164000.0,...,169500.0,194000.0,157000.0,192000.0,168000.0,169000.0,175000.0,200000.0,184100.0,183000.0
117,Kerrville,TX,255000.0,215000.0,235000.0,253125.0,250000.0,266000.0,257500.0,223500.0,...,239750.0,256750.0,268000.0,273500.0,278500.0,275079.0,304500.0,265650.0,281063.0,298750.0
118,Shelbyville,TN,159900.0,160000.0,155500.0,160000.0,166800.0,164000.0,162500.0,171000.0,...,191500.0,205000.0,187495.0,174950.0,189000.0,189950.0,209150.0,187000.0,213000.0,198000.0


In [132]:
#renaming column name and selecting wanted data from rental.csv
rental_df = rental_df.rename(columns={"RegionName":"City"})
rental_df.drop(rental_df.iloc[:, 2:50], inplace = True, axis = 1) 
del rental_df ["RegionID"]

#spliting state name form city and adding in a new column
rental_df["State"] = rental_df["City"].str.split(',').str[1]
rental_df['City'] = rental_df['City'].str.split(',').str[0]

#rearrnaging the order of the colums 
columns = ['City', 'State','2018-01','2018-02', '2018-03', '2018-04', '2018-05', '2018-06','2018-07', 
           '2018-08','2018-09', '2018-10', '2018-11', '2018-12','2019-01', '2019-02', '2019-03', 
           '2019-04', '2019-05', '2019-06','2019-07', '2019-08', '2019-09', '2019-10', '2019-11', 
           '2019-12','2020-01', '2020-02', '2020-03', '2020-04','2020-05', '2020-06','2020-07', 
           '2020-08', '2020-09']

rental_df=rental_df.loc[:, columns]

#renaming columns and data
rental_df = rental_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

#mering with common city list 
rental_df = pd.merge(pop_df["City"], rental_df, on="City", how="outer")
new_rental_df=rental_df.dropna(how="any")
new_rental_df.reset_index(drop=True, inplace=True)
new_rental_df.to_csv("cleaned_rental.csv")
new_rental_df

,City,State,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09
0,New York,NY,2598.0,2601.0,2605.0,2608.0,2612.0,2617.0,2621.0,2626.0,...,2698.0,2694.0,2686.0,2678.0,2671.0,2662.0,2653.0,2644.0,2635.0,2625.0
1,Los Angeles,CA,2380.0,2388.0,2397.0,2405.0,2414.0,2423.0,2432.0,2440.0,...,2542.0,2544.0,2545.0,2545.0,2546.0,2546.0,2546.0,2546.0,2545.0,2545.0
2,Chicago,IL,1705.0,1708.0,1710.0,1713.0,1716.0,1719.0,1722.0,1725.0,...,1778.0,1778.0,1777.0,1776.0,1776.0,1775.0,1774.0,1772.0,1771.0,1770.0
3,Dallas,TX,1470.0,1474.0,1478.0,1481.0,1485.0,1489.0,1493.0,1498.0,...,1560.0,1562.0,1563.0,1564.0,1566.0,1567.0,1568.0,1570.0,1571.0,1572.0
4,Philadelphia,PA,1505.0,1509.0,1512.0,1516.0,1521.0,1525.0,1530.0,1534.0,...,1598.0,1600.0,1602.0,1604.0,1606.0,1608.0,1610.0,1612.0,1614.0,1615.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Port St. Lucie,FL,1486.0,1495.0,1503.0,1512.0,1520.0,1529.0,1538.0,1545.0,...,1640.0,1648.0,1656.0,1664.0,1671.0,1679.0,1687.0,1695.0,1702.0,1710.0
101,Fort Collins,CO,1541.0,1545.0,1550.0,1555.0,1559.0,1564.0,1569.0,1574.0,...,1636.0,1638.0,1641.0,1644.0,1646.0,1649.0,1651.0,1653.0,1656.0,1658.0
102,Boulder,CO,1884.0,1890.0,1895.0,1901.0,1906.0,1911.0,1916.0,1921.0,...,1991.0,1992.0,1993.0,1995.0,1996.0,1997.0,1998.0,1998.0,1999.0,2000.0
103,Greeley,CO,1568.0,1574.0,1580.0,1586.0,1592.0,1598.0,1604.0,1611.0,...,1680.0,1682.0,1685.0,1688.0,1690.0,1693.0,1695.0,1698.0,1700.0,1703.0
